In [1]:
# Clean up in case of re-running
ManyToOneChildProtect.objects.all().delete()
ManyToOneParent.objects.all().delete()
ManyToOneChildCascade.objects.all().delete()
ManyToOneChildNull.objects.all().delete();

In [2]:
# Populate a parent with all 3 kinds of children.
parent = ManyToOneParent.objects.create()

[ManyToOneChildCascade.objects.create(parent=parent) for _ in range(2)]
[ManyToOneChildNull.objects.create(parent=parent) for _ in range(2)]
ManyToOneChildProtect.objects.create(parent=parent)

[p.print_relations() for p in ManyToOneParent.objects.all()];

ManyToOneParent id: 1 has cascade children ['ManyToOneChildCascade 2', 'ManyToOneChildCascade 3']
ManyToOneParent id: 1 has null children ['ManyToOneChildNull 4', 'ManyToOneChildNull 5']
ManyToOneParent id: 1 has protect children ['ManyToOneChildProtect 6']


In [3]:
# Deletion behavior of the child does not depend on the type.
parent.children_cascade.last().delete()
parent.children_null.last().delete()
parent.children_protect.last().delete()

[p.print_relations() for p in ManyToOneParent.objects.all()];

Delete called on ManyToOneChildCascade 3
Delete called on ManyToOneChildNull 5
Delete called on ManyToOneChildProtect 6
ManyToOneParent id: 1 has cascade children ['ManyToOneChildCascade 2']
ManyToOneParent id: 1 has null children ['ManyToOneChildNull 4']
ManyToOneParent id: 1 has protect children []


In [4]:
# But when deleting the parent, the behavior depends on on_delete in the child ForeignKey to the parent
def get_ids_parents(ModelClass):
    return [t for t in ModelClass.objects.values_list("id", "parent", named=True)]

print('Existing children')
print(f'ManyToOneChildCascade: {get_ids_parents(ManyToOneChildCascade)}')
print(f'ManyToOneChildNull: {get_ids_parents(ManyToOneChildNull)}')

print('\nDeleting the parent of these children\n')
parent.delete()

# Here be the dragon!
print(f'ManyToOneChildCascade: {get_ids_parents(ManyToOneChildCascade)} <--Got deleted with the parent!')
print(f'ManyToOneChildNull: {get_ids_parents(ManyToOneChildNull)} <-- Orphan :_( ');

Existing children
ManyToOneChildCascade: [Row(id=2, parent=1)]
ManyToOneChildNull: [Row(id=4, parent=1)]

Deleting the parent of these children

Delete called on ManyToOneParent 1
ManyToOneChildCascade: [] <--Got deleted with the parent!
ManyToOneChildNull: [Row(id=4, parent=None)] <-- Orphan :_( 


In [5]:
# Now the final case - Try to delete a parent protected by a child
from django.db.models import ProtectedError

parent = ManyToOneParent.objects.create()
ManyToOneChildProtect.objects.create(parent=parent)

try:
    parent.delete()
    print('\nThe parent was deleted!\n')
except ProtectedError:
    print('\nThe parent could not be deleted!\n')
    parent.print_relations()

Delete called on ManyToOneParent 7

The parent could not be deleted!

ManyToOneParent id: 7 has cascade children []
ManyToOneParent id: 7 has null children []
ManyToOneParent id: 7 has protect children ['ManyToOneChildProtect 8']


In [6]:
# The child is protecting the parent as long as it exists.  One use of this is to enforce clean-up,
# there can be no orhpaned child with a protected parent.
parent.children_protect.first().delete()
parent.delete();

Delete called on ManyToOneChildProtect 8
Delete called on ManyToOneParent 7
